In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-2").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
home_improv_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"), sep="\t", header=True)
home_improv_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

# Filter by Votes

In [8]:
filtered_home_improv_df = home_improv_df.filter("total_votes == 0").select(["star_rating","helpful_votes","total_votes","vine","verified_purchase"])
filtered_home_improv_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [10]:
# Filter for greater than 20 total votes
total_votes_df = home_improv_df.filter("total_votes > 20").select(["star_rating","helpful_votes","total_votes","vine","verified_purchase"])
total_votes_df.show(10)


+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|           21|         21|   N|                Y|
|          5|           43|         45|   N|                Y|
|          4|           63|         71|   N|                Y|
|          5|           28|         29|   N|                Y|
|          1|           32|         35|   N|                Y|
|          4|          106|        115|   N|                Y|
|          5|           33|         34|   N|                Y|
|          1|           91|         92|   N|                Y|
|          5|           23|         23|   N|                N|
|          4|           86|         91|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|           21|         21|   N|                Y|
|          5|           43|         45|   N|                Y|
|          4|           63|         71|   N|                Y|
|          5|           28|         29|   N|                Y|
|          1|           32|         35|   N|                Y|
|          4|          106|        115|   N|                Y|
|          5|           33|         34|   N|                Y|
|          1|           91|         92|   N|                Y|
|          5|           23|         23|   N|                N|
|          4|           86|         91|   N|                N|
|          5|           39|         45|   N|                Y|
|          5|           21|         22|   N|                N|
|          5|           27|         27|   N|           

In [16]:
# Filter for greater than 50% helpful vote percentage

helpful_votes_df=home_improv_df.filter("helpful_votes/total_votes > .5 and total_votes > 20").select(["star_rating","helpful_votes","total_votes","vine","verified_purchase"])
helpful_votes_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|           21|         21|   N|                Y|
|          5|           43|         45|   N|                Y|
|          4|           63|         71|   N|                Y|
|          5|           28|         29|   N|                Y|
|          1|           32|         35|   N|                Y|
|          4|          106|        115|   N|                Y|
|          5|           33|         34|   N|                Y|
|          1|           91|         92|   N|                Y|
|          5|           23|         23|   N|                N|
|          4|           86|         91|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



# Describe Stats

In [23]:
# Paid reviews
paid_df=home_improv_df.filter("vine == 'Y'").select(["star_rating","helpful_votes","total_votes","vine","verified_purchase"])
paid_df.describe().show(10)


+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             10779|             10779|             10779|10779|            10779|
|   mean| 4.305965302903794|  2.65033862139345|3.2816587809629834| null|             null|
| stddev|0.8854887968019488|15.899605954216419| 17.10616778907542| null|             null|
|    min|                 1|                 0|                 0|    Y|                N|
|    max|                 5|                96|                99|    Y|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



In [24]:
# Unpaid reviews

unpaid_df=home_improv_df.filter("vine == 'N'").select(["star_rating","helpful_votes","total_votes","vine","verified_purchase"])
unpaid_df.describe().show(10)


+-------+------------------+------------------+------------------+-------+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|
+-------+------------------+------------------+------------------+-------+-----------------+
|  count|           2623758|           2623758|           2623758|2623758|          2623758|
|   mean| 4.181868525984485| 1.678327040832272|2.0215397151719023|   null|             null|
| stddev|1.3039638701691976|15.910621255078016|16.761139850927773|   null|             null|
|    min|                 1|                 0|                 0|      N|                N|
|    max|                 5|               991|               996|      N|                Y|
+-------+------------------+------------------+------------------+-------+-----------------+



# Determine the percentage of five-star review among vine reviews

In [30]:
#paid_df.count()
paid_number=paid_df.count()
#paid_df.filter("star_rating==5").count()
paid_five_star_number=paid_df.filter("star_rating==5").count()
percentage_five_star_vine = paid_five_star_number/paid_number
#print(percentage_five_star_vine)
print(f'Number of paid reviews {paid_number}')
print(f'Number of paid five star reviews {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_vine * 100}%')

Number of paid reviews 10779
Number of paid five star reviews 5557
Percantage of paid reviews that are five stars 51.55394749049077%


# Determine the percentage of five-star review among non-Vine reviews

In [36]:
#unpaid_df.count()
unpaid_number=unpaid_df.count()
#unpaid_df.filter("star_rating==5").count()
unpaid_five_star_number=unpaid_df.filter("star_rating==5").count()
percentage_five_star_non_vine = unpaid_five_star_number/unpaid_number
print(f'Number of unpaid reviews {unpaid_number}')
print(f'Number of unpaid five star reviews {unpaid_five_star_number}')
print(f'Percantage of unpaid reviews that are five stars {percentage_five_star_non_vine * 100}%')

Number of unpaid reviews 2623758
Number of unpaid five star reviews 1650924
Percantage of unpaid reviews that are five stars 62.9221140059411%


**Additional analysis - check if the percentage pattern is different for the voted reviews that are paid and non paid**

In [35]:
#paid_df.count()
paid_number=paid_df.count()
#paid_df.filter("star_rating==5").count()
paid_voted_five_star_number=paid_df.filter("star_rating==5 and total_votes > 0").count()
percentage_voted_five_star_vine = paid_voted_five_star_number/paid_number
#print(percentage_five_star_vine)
print(f'Number of paid reviews with votes {paid_number}')
print(f'Number of paid voted five star reviews with votes {paid_voted_five_star_number}')
print(f'Percantage of paid reviews that are five stars with votes {percentage_voted_five_star_vine * 100}%')

Number of paid reviews with votes 10779
Number of paid voted five star reviews with votes 2899
Percantage of paid reviews that are five stars with votes 26.89488820855367%


In [37]:
#unpaid_df.count()
unpaid_number=unpaid_df.count()
#unpaid_df.filter("star_rating==5").count()
unpaid_voted_five_star_number=unpaid_df.filter("star_rating==5 and total_votes >0").count()
percentage_voted_five_star_non_vine = unpaid_voted_five_star_number/unpaid_number
print(f'Number of unpaid reviews {unpaid_number}')
print(f'Number of unpaid voted five star reviews {unpaid_voted_five_star_number}')
print(f'Percantage of unpaid voted reviews that are five stars {percentage_voted_five_star_non_vine * 100}%')

Number of unpaid reviews 2623758
Number of unpaid voted five star reviews 536382
Percantage of unpaid voted reviews that are five stars 20.443272588401825%
